In [ ]:
%pip install pymilvus SentenceTransformer

In [7]:
from dotenv import load_dotenv
load_dotenv()
import os
from pymilvus import (connections,
                      utility,
                      FieldSchema,
                      CollectionSchema,
                      DataType,
                      Collection,
                      MilvusClient)
collection_name = "test_connection"
# milvus_client = MilvusClient(
#                                     uri= "http://localhost:19530",
#                                     token = "root:Milvus")
connections.connect('default',host=os.getenv("MILVUS_HOST","localhost"),port=os.getenv("MILVUS_PORT","19530"))

milvus_client = MilvusClient(
                                    uri= os.getenv("MILVUS_URI"),
                                    token = "root:Milvus")
print(f"{collection_name}---------------collection has --------------------")
milvus_client.load_collection(collection_name = collection_name)
milvus_client

2025-03-20 09:06:54,272 [ERROR][handler]: RPC error: [load_collection], <MilvusException: (code=100, message=collection not found[database=default][collection=test_connection])>, <Time:{'RPC start': '2025-03-20 09:06:54.265371', 'RPC error': '2025-03-20 09:06:54.272626'}> (decorators.py:140)
2025-03-20 09:06:54,273 [ERROR][load_collection]: Failed to load collection: test_connection (milvus_client.py:960)


test_connection---------------collection has --------------------


MilvusException: <MilvusException: (code=100, message=collection not found[database=default][collection=test_connection])>

In [ ]:



connections.connect('default',host="localhost",port="19530")


fields = [
    FieldSchema(name='pk', dtype=DataType.INT64, is_primary=True, auto_id= True),
    FieldSchema(name='random', dtype=DataType.DOUBLE),
    FieldSchema(name='embeddings', dtype=DataType.FLOAT_VECTOR,dim=768),
    FieldSchema(name='text', dtype=DataType.VARCHAR , max_length = 500),
    # FieldSchema(name='filename', dtype=DataType.VARCHAR , max_length = 500),
    # FieldSchema(name='categorie', dtype=DataType.VARCHAR , max_length = 500),
]


schema = CollectionSchema(fields,"this is aks text")

milvusdb = Collection(collection_name,schema)



In [94]:
import random

In [95]:
from sentence_transformers import SentenceTransformer


texts = [
    "this is a first test case on the milvus db where it doesn't support windows for milvus lite",
    "this is the second test case on milvus where you can use milvus standalone in windows. but failed due to wsl 2 distro setting config",
    "this is the third test case on milvus using it on vmware with kali linux idk how it works but i gave up on it in half way.",
    "this is the forth test case on milvus using on windows with standalone in docker run successfuly on sample code by milvus docs.",
    "this is the fifth test case on  milus using standalone via docker to store text and embeddings in the milvus db. idk how it will turn out coz iam still not done yet."
]

text_embed = []
model = SentenceTransformer('all-mpnet-base-v2')
for t in texts:
    text_embed.append(model.encode(t))

print(len(text_embed[0]))



768


In [96]:
insert_data = [
    [i for i in range(len(texts))],
    [float(random.randrange(-20, -10)) for _ in range(len(texts))],
    [emb for emb in text_embed],
    [text for text in texts]
]

In [97]:
inserting = milvusdb.insert(insert_data)
milvusdb.flush()

In [101]:
# index = {
#     "index_type": "IVF_FLAT",
#     "metric_type": "L2",
#     "params": {"nlist": 128},
# }

index = {
    "index_type": "HNSW",
    "metric_type": "L2",
    "params": {"M": 16, "efConstruction": 200}
}

In [104]:
milvusdb.create_index("embeddings", index)

Status(code=0, message=)

In [ ]:
milvusdb.load()
search_text = "which test case did the following milus using standalone via docker to store text"
search_embed = model.encode(search_text)


In [106]:
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 100},
}

In [107]:
result = milvusdb.search([search_embed],"embeddings",search_params, limit=5, output_fields=["text"])
result

data: ['["pk: 4, distance: 0.43936729431152344, entity: {\'text\': \'this is the fifth test case on  milus using standalone via docker to store text and embeddings in the milvus db. idk how it will turn out coz iam still not done yet.\'}", "pk: 3, distance: 0.7304867506027222, entity: {\'text\': \'this is the forth test case on milvus using on windows with standalone in docker run successfuly on sample code by milvus docs.\'}", "pk: 2, distance: 1.1145399808883667, entity: {\'text\': \'this is the third test case on milvus using it on vmware with kali linux idk how it works but i gave up on it in half way.\'}", "pk: 1, distance: 1.2889277935028076, entity: {\'text\': \'this is the second test case on milvus where you can use milvus standalone in windows. but failed due to wsl 2 distro setting config\'}", \'pk: 0, distance: 1.4334330558776855, entity: {\\\'text\\\': "this is a first test case on the milvus db where it doesn\\\'t support windows for milvus lite"}\']']

In [110]:
import json
for res in result:
    for i in res:
        print(i)

pk: 4, distance: 0.43936729431152344, entity: {'text': 'this is the fifth test case on  milus using standalone via docker to store text and embeddings in the milvus db. idk how it will turn out coz iam still not done yet.'}
pk: 3, distance: 0.7304867506027222, entity: {'text': 'this is the forth test case on milvus using on windows with standalone in docker run successfuly on sample code by milvus docs.'}
pk: 2, distance: 1.1145399808883667, entity: {'text': 'this is the third test case on milvus using it on vmware with kali linux idk how it works but i gave up on it in half way.'}
pk: 1, distance: 1.2889277935028076, entity: {'text': 'this is the second test case on milvus where you can use milvus standalone in windows. but failed due to wsl 2 distro setting config'}
pk: 0, distance: 1.4334330558776855, entity: {'text': "this is a first test case on the milvus db where it doesn't support windows for milvus lite"}
